In [1]:
%load_ext autoreload
%autoreload 2

In [24]:
from collections import Counter
import itertools
import random

from jaccard import *
from utils import *

In [15]:
np.set_printoptions(linewidth=1000, suppress=True)

In [4]:
meta, projects, votes = load_pb_ohe('data/poland_warszawa_2023_srodmiescie.pb')

In [5]:
n = meta['num_votes']
m = meta['num_projects']
assert votes.shape == (n, m)
costs = np.array(projects['cost'].astype('float')) / meta['budget']

In [6]:
def cost(center: np.ndarray) -> float:
    return (center * costs).sum()

In [7]:
def dist_to_centers(centers: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    d = jaccard(votes, centers)
    return d.min(axis=1), d.argmin(axis=1)

In [8]:
def remove_bad_centers(centers: np.ndarray) -> np.ndarray:
    while True:
        _, assignment = dist_to_centers(centers)
        if len(centers) == 1:
            break
        efficiency = [
            ((n2 / n) / cost(centers[c]), c)
            for c, n2 in zip(*np.unique(assignment, return_counts=True))
        ]
        eff, c = min(efficiency)
        if eff < 1.0:
            centers = np.delete(centers, c, axis=0)
            #print(f"Removing center that has efficiency {eff}. Centers: {len(centers)}")
        else:
            break
    return centers

In [83]:
def is_blocking_coalition(old: np.ndarray, center: np.ndarray) -> bool:
    new = jaccard(votes, center.reshape(1, m)).reshape(-1)
    return np.sum(new < old) / n >= cost(center)

candidates = np.concatenate([
    np.eye(m),
    [a+b for a, b in itertools.combinations(np.eye(m), 2)],
    [a+b+c for a, b, c in itertools.combinations(np.eye(m), 3)],
    #[a+b+c+d for a, b, c, d in itertools.combinations(np.eye(m), 4)],
])
linspace = 1 - np.linspace(0, 1, 10, endpoint=False)

def best_blocking_coalition(old: np.ndarray, center: np.ndarray) -> np.ndarray | None:
    new = jaccard(votes, center.reshape(1, m)).reshape(-1)
    coalition = new < old
    if np.sum(coalition) / n < cost(center):
        return None
    #return center
    V = votes[coalition]
    grad = gradient(center, V)
    e_dist = np.e ** (10 * (new[coalition] - old[coalition]).reshape(-1, 1))
    center2 = best_blocking_coalition(old, np.clip(center + 0.05 * (e_dist / e_dist.sum() * grad).sum(axis=0), 0, 1))
    return center2 if center2 is not None else center

def find_blocking_coalition(centers: np.ndarray) -> np.ndarray | None:
    old, _ = dist_to_centers(centers)
    for v in candidates:
        for a in linspace:
            """
            u = a * v
            assert u.sum() > 0.0
            if is_blocking_coalition(old, u):
                return u
            """
            u = best_blocking_coalition(old, a * v)
            if u is not None:
                return u
    return None

In [84]:
centers = remove_bad_centers(np.eye(m))
for i in range(1000):
    c = find_blocking_coalition(centers)
    if c is not None:
        centers = np.append(centers, c.reshape(1, m), axis=0)
        print(f"{i=:<4}  centers={len(centers):<2}  blocking coalition: {c}")
        centers = remove_bad_centers(centers)
    else:
        break
else:
    print(f"Did not converge!")

i=0     centers=12  blocking coalition: [0.58845137 0.6129949  0.54882017 0.5251014  0.47305148 0.44351518 0.47825872 0.3616081  0.6364366  0.29303417 0.45235986 0.44141933 0.31734028 0.43554566 0.51635232 0.29324174 0.95476006 0.2019184  0.77464751 0.12550273 0.16120622 0.14979485 0.16137083 0.21791754 0.70555884 0.21642432 0.09423912 0.0857159  0.0332334  0.04393718 0.00744747 0.05867626 0.00028486 0.01144913 0.27847634 0.00003562 0.         0.00024634 0.00004629 0.00006962 0.00010038 0.00005025 0.         0.         0.         0.00016172 0.        ]
i=1     centers=10  blocking coalition: [0.         0.         0.         0.         0.         0.         0.         0.         0.17450807 0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.  

In [85]:
_, assignment = dist_to_centers(centers)
for k, v in Counter(assignment).items():
    print(f"{k:>2}: {v/n:.2%}")

19: 63.56%
 8: 5.36%
 6: 1.52%
10: 7.29%
14: 0.21%
 0: 2.50%
 3: 4.22%
 1: 2.50%
17: 1.65%
 2: 1.65%
11: 0.54%
18: 1.78%
15: 2.76%
21: 0.88%
22: 0.49%
20: 0.46%
 4: 1.11%
 7: 0.64%
 5: 0.67%
 9: 0.15%
16: 0.08%


In [86]:
with np.printoptions(threshold=2*n, linewidth=20*m, precision=3, suppress=True):
    print(centers)

[[0.    0.    0.    0.    1.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    1.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    1.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.    1.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   

In [235]:
centers = remove_bad_centers(np.eye(m))
dist, assignment = dist_to_centers(centers)

In [240]:
lr = 0.001
for i in range(1000):
    dist, assignment = dist_to_centers(centers)
    print(f"{i=:<5} {lr=:<.3f} sum={dist.sum()}")
    gradient_descent(centers, lr)
    """
    l = list(range(n))
    random.shuffle(l)
    for a in l:
        b = assignment[a]
        centers[b] = np.clip(centers[b] + lr * gradient(centers[b], votes[a]), 0, 1)
    """

i=0     lr=0.001 sum=2238.1554159433376
i=1     lr=0.001 sum=2098.1434981866887
i=2     lr=0.001 sum=2101.65785120724
i=3     lr=0.001 sum=2081.4310900953037
i=4     lr=0.001 sum=2091.03751713349
i=5     lr=0.001 sum=2074.44556553625
i=6     lr=0.001 sum=2086.328261815478
i=7     lr=0.001 sum=2071.68283625815
i=8     lr=0.001 sum=2084.5985322564543
i=9     lr=0.001 sum=2069.995068887906
i=10    lr=0.001 sum=2083.7157955688144
i=11    lr=0.001 sum=2069.23651144583
i=12    lr=0.001 sum=2082.8765813003843
i=13    lr=0.001 sum=2068.811910813442
i=14    lr=0.001 sum=2082.439141016772
i=15    lr=0.001 sum=2068.1460506921994
i=16    lr=0.001 sum=2082.3304608906965
i=17    lr=0.001 sum=2068.083040726879
i=18    lr=0.001 sum=2081.868563328954
i=19    lr=0.001 sum=2068.1585400395434
i=20    lr=0.001 sum=2082.0793641160744
i=21    lr=0.001 sum=2067.7831573196872
i=22    lr=0.001 sum=2082.080889299559
i=23    lr=0.001 sum=2067.967791080039
i=24    lr=0.001 sum=2081.737838935922
i=25    lr=0.001 su

KeyboardInterrupt: 

In [197]:
idx = 10
center = centers[idx]
V = votes[assignment == idx]

In [200]:
mn = np.sum(np.minimum(center, V), axis=1)
mx = np.sum(np.maximum(center, V), axis=1)
print(mn.shape, mx.shape)
print(V.shape, center.shape)
print((V > center).shape)
grad = ((V > center) * mx.reshape(-1, 1) - (V < center) * mn.reshape(-1, 1)) / (mx ** 2).reshape(-1, 1)
print(grad.sum(axis=0).shape)

(54,) (54,)
(54, 47) (47,)
(54, 47)
(47,)


In [ ]:
voter = V[0]
mn = np.sum(np.minimum(voter, c))
mx = np.sum(np.maximum(voter, c))
(((voter > c) * mx - (voter < c) * mn) / (mx ** 2)).shape

(47,)

In [91]:
x = np.concatenate([
    np.eye(m),
    [a+b for a, b in itertools.combinations(np.eye(m), 2)],
    [a+b+c for a, b, c in itertools.combinations(np.eye(m), 3)],
    [a+b+c+d for a, b, c, d in itertools.combinations(np.eye(m), 4)],
])
print(len(x))
x = np.array([
    y for y in x if cost(y) <= 1
])
print(len(x))

195708
195708


In [119]:
a = m
def find(x: np.ndarray):
    l = len(x)
    if np.sum(x * costs[:l]) > 0.1:
        return
    if l == a:
        yield x
    else:
        yield from find(np.append(x, [0]))
        yield from find(np.append(x, [1]))
len(list(find(np.array([]))))

6000504

In [112]:
costs

array([0.10150436, 0.03022345, 0.03729565, 0.07545114, 0.01459582, 0.02076519, 0.07609602, 0.00571795, 0.02646164, 0.02429054, 0.03047065, 0.0152192 , 0.09651727, 0.02783739, 0.06233855, 0.19905344, 0.0566034 , 0.03783305, 0.16766921, 0.00444259, 0.02132408, 0.01903475, 0.08368198, 0.13649993, 0.1225275 , 0.18121172, 0.02575227, 0.00644882, 0.01006015, 0.03331888, 0.00214961, 0.01160787, 0.00429921, 0.00629082, 0.15047236, 0.0185081 , 0.02149605, 0.00429921, 0.00257953, 0.02891219, 0.00214961, 0.02712802, 0.0157652 , 0.00601889, 0.00859842, 0.03224408, 0.00644882])